### RFC

### import modules and configure notebook

In [4]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
import matplotlib.pyplot
import pickle

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)


from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

import xgboost as xgb

%matplotlib inline

### Load variables stored by data_preproccessing notebook

In [5]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats



### configurations
* save_plots -> True|False
* random_seed_state -> number, sets random state for model and for stratified splits 
* classify_bedrock_only -> True|False
* pickle_model -> True|False, wether model should be serialised and saved
* pickle_model_name -> string, name of serialised model
* grid_search -> True|False, if set to true then grid search is performed to identify optimum hyperparamaters for model 
* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1
* pickle_file_path -> string,  filepath for serialised model to be saved to

In [6]:
save_plots = False
random_seed_state = 42
classify_bedrock_only = False
pickle_model = False
pickle_model_name = 'grouped'
pickle_file_path = '../../../model'
grid_search = True
scale = False

### if only bedrock sites are classified then classes are label encoded, if bedrock sites alone are not being classified then the class sites would have already been label encoded in the 1 data_preproccessing notebook 

In [7]:
if classify_bedrock_only:
    train_data_formodel['class'], uniques = pd.factorize(train_data_formodel['class'])
    train_data_formodel = train_data_formodel[train_data_formodel['Geology']=='Bedrock']

### The class column is stored as the variable y 

In [8]:
y = np.array(train_data_formodel['class'])

### The variables identified as best by the 2 feature_selection notebook are used as features

In [9]:
train_data_feats = train_data_formodel[best_feats]

In [10]:
if scale:
    my_scaler = StandardScaler()
    X = np.array(my_scaler.fit_transform(train_data_feats))
else:
    X = np.array(train_data_feats)

### the dimensions of the class and features are checked

In [11]:
print(X.shape)
print(y.shape)

(1158, 31)
(1158,)


### Gri dsearch is done to identify best hyperparamaters for the model

In [15]:
%%time
if grid_search:
    esti = xgb.XGBClassifier(booster = 'gbtree')


    
    n_estimators = [int(x) for x in np.linspace(start = 4000, stop = 8000, num = 4)]
    max_depth = [int(x) for x in np.linspace(10, 60, num = 3)]
    min_child_weight = [0,3]
    gamma = [0,3]

    
    
    random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
                   'min_child_weight':min_child_weight,
                   'gamma':gamma
                  }
    
    clf = RandomizedSearchCV(estimator = esti, n_iter=20, param_distributions = random_grid,
                              n_jobs=-1, scoring='f1_macro', cv = 3, verbose=3, 
                              random_state = random_seed_state)
    
    clf.fit(X, y)
    print(clf.best_params_)
mistake

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] n_estimators=8000, min_child_weight=0, max_depth=10, gamma=3 ....
[CV] n_estimators=8000, min_child_weight=0, max_depth=10, gamma=3 ....
[CV] n_estimators=8000, min_child_weight=0, max_depth=10, gamma=3 ....
[CV] n_estimators=4000, min_child_weight=0, max_depth=60, gamma=3 ....


/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  n_estimators=4000, min_child_weight=0, max_depth=60, gamma=3, score=0.45297786004901325, total= 4.0min
[CV] n_estimators=4000, min_child_weight=0, max_depth=60, gamma=3 ....


/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future t

[CV]  n_estimators=8000, min_child_weight=0, max_depth=10, gamma=3, score=0.4549037141035065, total= 7.7min
[CV] n_estimators=4000, min_child_weight=0, max_depth=60, gamma=3 ....


...

/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  n_estimators=5333, min_child_weight=0, max_depth=35, gamma=0, score=0.5424098089326479, total=15.3min


[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 148.8min finished


{'n_estimators': 6666, 'min_child_weight': 3, 'max_depth': 10, 'gamma': 0}


NameError: name 'mistake' is not defined

best paramaters  from the randomised grid search were {'n_estimators': 6666, 'min_child_weight': 3, 'max_depth': 10, 'gamma': 0}, took 149 mins to fit 60 models

In [ ]:
if grid_search:
    esti = xgb.XGBClassifier(booster = 'gbtree')

    n_estimators = [5000, 6000, 7000]
    min_child_weight = [3, 5, 7]
    max_depth = [5,10,15 ]

    
    
    random_grid = {'n_estimators': n_estimators,
               'min_child_weight': min_child_weight,
               'gamma': gamma,
                  'max_depth':max_depth
                  }
    
    clf = GridSearchCV(estimator = esti, param_grid = random_grid,
                              n_jobs=-1, scoring='f1_macro', cv = 3, verbose=3 
                              )
    clf.fit(X, y)
    print(clf.best_params_)
mistake

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] gamma=0, max_depth=5, min_child_weight=3, n_estimators=5000 .....
[CV] gamma=0, max_depth=5, min_child_weight=3, n_estimators=5000 .....
[CV] gamma=0, max_depth=5, min_child_weight=3, n_estimators=5000 .....
[CV] gamma=0, max_depth=5, min_child_weight=3, n_estimators=6000 .....


/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

[CV]  gamma=0, max_depth=5, min_child_weight=3, n_estimators=5000, score=0.5837434585011825, total=  50.6s
[CV] gamma=0, max_depth=5, min_child_weight=3, n_estimators=6000 .....


/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  gamma=0, max_depth=5, min_child_weight=3, n_estimators=5000, score=0.6882162668268137, total=  51.2s
[CV] gamma=0, max_depth=5, min_child_weight=3, n_estimators=6000 .....


...

[CV]  gamma=1, max_depth=10, min_child_weight=7, n_estimators=6000, score=0.5373329768959451, total= 8.8min
[CV] gamma=1, max_depth=10, min_child_weight=7, n_estimators=7000 ....


/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  gamma=1, max_depth=10, min_child_weight=7, n_estimators=6000, score=0.6615972191002935, total= 9.2min
[CV] gamma=1, max_depth=10, min_child_weight=7, n_estimators=7000 ....


/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  gamma=1, max_depth=10, min_child_weight=7, n_estimators=6000, score=0.5603775111690206, total= 8.9min
[CV] gamma=1, max_depth=15, min_child_weight=3, n_estimators=5000 ....


In [ ]:
best paramaters from the grid search were {'gamma': 0, 'min_child_weight': 7, 'n_estimators': 5000}


### Carry out 10-f0ld stratified cross validation, class f1 scores and macro f1 scores with weighted averages are calculated

In [ ]:
esti = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=10, random_state=random_seed_state)
skf.get_n_splits(X, y)
class_f1_scores = []
macro_f1_scores = []
accuracy_scores = []
feat_imp =[]
f1_dict = {}
feat_imp_dict = {}
count = 0
for train_index, test_index in skf.split(X, y):
    count = count + 1
    print('making model:')
    key = 'round' + str(count)
    print(count)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    esti.fit(X_train, y_train)
    y_pred = esti.predict(X_test)
    class_f1_scores = f1_score(y_test, y_pred, average = None)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    macro_f1_scores.append(f1_score(y_test, y_pred, average = 'weighted'))
    f1_dict[key] = class_f1_scores 
    feat_imp_dict[key] = esti.feature_importances_

In [ ]:
f1_df = pd.DataFrame(data = f1_dict)


In [ ]:
for key in f1_dict:
    print(len(f1_dict[key]))

### Below are the encodings for the class variable

In [ ]:
print(train_data_formodel['class'].unique())
print(list(uniques))

In [ ]:
f1_df_final = pd.concat([f1_df, pd.Series(uniques)], axis = 1)

In [ ]:
f1_df_final.rename(columns={0:'class'}, inplace=True)
f1_df_final.set_index('class', drop = True, inplace = True)

### Boxplot showing the distribution of class f1 scores from 10 models

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
plot = sns.boxplot(data = f1_df_final.T)
plot.set_title('F1 scores for each site', fontdict={'fontsize': 14})
plot.set_ylabel('F1 score', fontdict={'fontsize': 11})
plot.set_xlabel("Bedrock site or superficial site", fontdict={'fontsize': 11})

if save_plots == True:
    fig = plot.get_figure()
    fig.savefig('output/site_specific_f1_scores.png')

In [ ]:
pd.DataFrame(data = f1_df_final.T.median()).to_csv('output/median_class_f1_scores.csv')

### Boxplot showing the macro F1 score with weighted averages

In [ ]:
plot = sns.boxplot(macro_f1_scores)
plot.set_title('Average-weighted macro-f1 score', fontdict={'fontsize': 14})
plot.set_xlabel("F1-score", fontdict={'fontsize': 11})

if save_plots == True:
    fig = plot.get_figure()
    fig.savefig('output/macro_f1_scores.png')

In [ ]:
pd.Series(pd.Series(macro_f1_scores).median()).to_csv('output/median_macro_f1.csv')

In [ ]:
pd.Series(macro_f1_scores).median()

### Boxplot showing accuracy scores

In [ ]:
sns.boxplot(accuracy_scores)

### Get feature importances

feat_imp_df = pd.DataFrame(data = feat_imp_dict)
feat_imp_df.head()

feat_imp_df_final = pd.concat([feat_imp_df, pd.Series(my_data[my_data.columns.values[9:-1]].columns.values)], axis = 1)
feat_imp_df_final.rename(columns = {0:'element'}, inplace = True )
feat_imp_df_final.head()

feat_imp_df_final.set_index('element', inplace=True)


feat_imp_df_final_plot = feat_imp_df_final.T

feat_imp_df_final_plot

elements = feat_imp_df_final_plot.columns.values 
mean_feature_importance = []
for col in list(feat_imp_df_final_plot.columns.values):
    mean_feature_importance.append(feat_imp_df_final_plot[col].mean())
    

mean_feature_importance_df = pd.concat([pd.Series(elements), pd.Series(mean_feature_importance)], axis = 1)

mean_feature_importance_df.rename(columns={0:'elements', 1:'mean_importance'}, inplace=True)

mean_feature_importance_df.sort_values(by='mean_importance', ascending=False, inplace=True)

ordered_col_names = list(mean_feature_importance_df['elements'])

sns.set_style("whitegrid")
sns.set_style()
sns.set(rc={'figure.figsize':(20,20)})
plot = sns.boxplot(data = feat_imp_df_final_plot[ordered_col_names])
plot.set_xticklabels(plot.get_xticklabels(),rotation=90, ha = 'left')
plot.set_title('Feature (element) importance', fontdict={'fontsize': 20})
plot.set_ylabel('Feature importance', fontdict={'fontsize': 15})
plot.set_xlabel("Element", fontdict={'fontsize': 15})

if save_plots == True:
    fig = plot.get_figure()
    fig.savefig('output/feature_importances.png')

### Model is built for predicting source of artefacts 

esti_final = xgb.XGBClassifier()

esti_final.fit(X, y)

In [ ]:
if pickle_model == True:
    pickle.dump(esti_final, open(pickle_file_path+'.sav', 'wb'))

print(train_data_formodel['class'].unique())
print(uniques)

df_for_identifiers = test_data.copy(deep = True)
identifiers =  df_for_identifiers['Analysis']

### Predictions are made for the artefacts

y_pred = esti_final.predict(np.array(test_data[test_data.columns.values[9:-1]]))

y_pred_proba = esti_final.predict_proba(np.array(test_data[test_data.columns.values[9:-1]]))


probabilities_df = pd.DataFrame(data = y_pred_proba, columns = uniques)
probabilities_df_final = pd.concat([probabilities_df, pd.Series(list(identifiers))], axis = 1)

probabilities_df_final.rename(columns = {0:'identifier'}, inplace=True)

final_pred_df = pd.concat([pd.Series(y_pred), probabilities_df_final], axis = 1)

final_pred_df.rename(columns={0:'class_number'}, inplace = True)

### predictions are outputted as csv file

final_pred_df.to_csv('output/predictions.csv')


uniques_list = list(uniques)
def get_pred_names(row):
    return(uniques_list[row['class_number']])
final_pred_df['class_predictions'] = final_pred_df.apply(get_pred_names, axis = 1)

final_pred_df.head()